In [7]:
import numpy as np
from qiskit import QuantumCircuit, Aer, execute
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize

# 示例参数
n = 2  # 操作数
m = 2  # 机器数
p = np.array([[1, 2], [3, 1]])  # p[i][j] 表示操作 i 在机器 j 上的处理时间
K1 = 10  # 惩罚系数 1
K2 = 10  # 惩罚系数 2
l = 3  # z 和 τ 的二进制位数（假设最大 makespan 为 2^3 - 1）

# 定义变量总数
num_x = n * m  # x_ij 变量
num_z = l  # z_h 变量
num_tau = m * l  # tau 变量
total_vars = num_x + num_z + num_tau

# 创建 QuadraticProgram 表示目标函数
qp = QuadraticProgram("FJSP")

# 添加二进制变量
for i in range(n):
    for j in range(m):
        qp.binary_var(f"x_{i}_{j}")
for h in range(l):
    qp.binary_var(f"z_{h}")
for j in range(m):
    for h in range(l):
        qp.binary_var(f"tau_{j}_{h}")

# 第一项：目标函数 ∑ 2^h z_h
linear_terms = {f"z_{h}": 2 ** h for h in range(l)}
qp.minimize(linear=linear_terms)

# 第二项：约束 K1 ∑_i (∑_j x_{ij} - 1)^2
for i in range(n):
    constraint = {f"x_{i}_{j}": 1 for j in range(m)}
    qp.linear_constraint(linear=constraint, sense="==", rhs=1, name=f"one_machine_{i}")

# 第三项：约束 K2 ∑_j (∑_i x_{ij} p_{ij} + ∑_h 2^h τ_{jh} - ∑_h 2^h z_h)^2
for j in range(m):
    linear = {f"x_{i}_{j}": p[i][j] for i in range(n)}
    linear.update({f"tau_{j}_{h}": 2 ** h for h in range(l)})
    linear.update({f"z_{h}": -2 ** h for h in range(l)})
    qp.linear_constraint(linear=linear, sense="==", rhs=0, name=f"machine_load_{j}")

# 转换为 QUBO
converter = QuadraticProgramToQubo()
qubo = converter.convert(qp)

# 提取哈密顿量
op, offset = qubo.to_ising()  # op 是 SparsePauliOp 对象

# 从 SparsePauliOp 中提取线性项和二次项系数
linear_coeffs = {}  # a_k for Z_k
quadratic_coeffs = {}  # b_{ij} for Z_i Z_j
for pauli, coeff in zip(op.paulis, op.coeffs):
    pauli_str = pauli.to_label()
    coeff = coeff.real
    if "Z" not in pauli_str:
        continue
    z_indices = [i for i, p in enumerate(pauli_str[::-1]) if p == "Z"]  # 反转索引以匹配 Qiskit 顺序
    if len(z_indices) == 1:
        linear_coeffs[z_indices[0]] = coeff
    elif len(z_indices) == 2:
        quadratic_coeffs[tuple(sorted(z_indices))] = coeff


# 定义 QAOA 电路构造函数
def create_qaoa_circuit(params, num_qubits, p_layers):
    gamma = params[:p_layers]  # 问题哈密顿量参数
    beta = params[p_layers:]  # 混合哈密顿量参数
    qc = QuantumCircuit(num_qubits)

    # 初始化均匀叠加态
    qc.h(range(num_qubits))

    # 应用 p 层 QAOA
    for p in range(p_layers):
        # 问题哈密顿量 H_P: RZ 和 RZZ 门
        for k, ak in linear_coeffs.items():
            qc.rz(2 * gamma[p] * ak, k)  # RZ(2γa_k) for e^{-iγa_k Z_k}
        for (i, j), bij in quadratic_coeffs.items():
            qc.rzz(2 * gamma[p] * bij, i, j)  # RZZ(2γb_{ij}) for e^{-iγb_{ij} Z_i Z_j}

        # 混合哈密顿量 H_M: RX 门
        for m in range(num_qubits):
            qc.rx(-2 * beta[p], m)  # RX(-2β) for e^{iβX_m}

    return qc


# 定义期望值计算函数
def compute_expectation(params, num_qubits, p_layers, operator):
    qc = create_qaoa_circuit(params, num_qubits, p_layers)
    backend = Aer.get_backend("statevector_simulator")
    job = execute(qc, backend)
    statevector = job.result().get_statevector()
    # 将 Statevector 转换为 NumPy 数组
    statevector_np = np.array(statevector)
    # 计算期望值 <ψ|H|ψ>
    h_matrix = operator.to_matrix()
    expectation = np.real(np.conj(statevector_np).T @ h_matrix @ statevector_np)
    return expectation + offset


# 优化目标函数
p_layers = 1
initial_params = np.random.uniform(0, np.pi, 2 * p_layers)  # [γ, β]
result = minimize(
    compute_expectation,
    initial_params,
    args=(total_vars, p_layers, op),
    method="COBYLA",
    options={"maxiter": 100}
)

# 输出结果
optimal_params = result.x
print("Optimal value:", result.fun)
print("Optimal parameters (γ, β):", optimal_params)

# 生成并打印最终电路
optimal_circuit = create_qaoa_circuit(optimal_params, total_vars, p_layers)
print(optimal_circuit)

C:\Users\李华宪\AppData\Local\Temp\ipykernel_47456\1703829948.py:99: DeprecationWarning: The function ``qiskit.execute_function.execute()`` is deprecated as of qiskit 0.46.0. It will be removed in the Qiskit 1.0 release. This function combines ``transpile`` and ``backend.run``, which is covered by ``Sampler`` :mod:`~qiskit.primitives`. Alternatively, you can also run :func:`.transpile` followed by ``backend.run()``.
  job = execute(qc, backend)


Optimal value: 194.01312923549835
Optimal parameters (γ, β): [3.0100966  0.96544606]
      ┌───┐┌─────────────┐                                                    »
 q_0: ┤ H ├┤ Rz(-96.323) ├─■────────────■─────────────────────────────────────»
      ├───┤├─────────────┤ │ZZ(24.081)  │                                     »
 q_1: ┤ H ├┤ Rz(-144.48) ├─■────────────┼────────────■────────────────────────»
      ├───┤├─────────────┤              │ZZ(72.242)  │                        »
 q_2: ┤ H ├┤ Rz(-288.97) ├──────────────■────────────┼────────────■───────────»
      ├───┤├─────────────┤                           │ZZ(48.162)  │ZZ(24.081) »
 q_3: ┤ H ├┤ Rz(-72.242) ├───────────────────────────■────────────■───────────»
      ├───┤└┬────────────┤                                                    »
 q_4: ┤ H ├─┤ Rz(165.56) ├────────────────────────────────────────────────────»
      ├───┤ ├────────────┤                                                    »
 q_5: ┤ H ├─┤ Rz(331.11) ├─────────

In [17]:
# 输出结果
# print("Optimal value:", result.eigenvalue.real + offset)
# print("Optimal parameters (γ, β):", result.optimal_parameters)
# optimal_circuit.measure(range(total_vars),range(total_vars))
optimal_circuit.measure_all()
backend = Aer.get_backend("qasm_simulator")
job = execute(optimal_circuit, backend, shots=1024)
result = job.result()
counts = result.get_counts()
# counts = job.result.get_counts(optimal_circuit)
print(counts)

{'0111011001011 0111011001011 0111011001011': 2, '0101111110011 0101111110011 0101111110011': 1, '1001001000011 1001001000011 1001001000011': 2, '0000111001110 0000111001110 0000111001110': 1, '1011010110000 1011010110000 1011010110000': 2, '1011001011011 1011001011011 1011001011011': 1, '1010111000001 1010111000001 1010111000001': 5, '1111010111111 1111010111111 1111010111111': 1, '1000010011110 1000010011110 1000010011110': 1, '0101111001100 0101111001100 0101111001100': 1, '0010111001010 0010111001010 0010111001010': 1, '1111000010111 1111000010111 1111000010111': 1, '0111000111100 0111000111100 0111000111100': 4, '0111111010110 0111111010110 0111111010110': 1, '0000011110101 0000011110101 0000011110101': 1, '0110011011100 0110011011100 0110011011100': 1, '0011001101011 0011001101011 0011001101011': 1, '0110110111011 0110110111011 0110110111011': 9, '0110110010111 0110110010111 0110110010111': 1, '0101011010010 0101011010010 0101011010010': 1, '0110010110010 0110010110010 0110010110

C:\Users\李华宪\AppData\Local\Temp\ipykernel_47456\3848296286.py:7: DeprecationWarning: The function ``qiskit.execute_function.execute()`` is deprecated as of qiskit 0.46.0. It will be removed in the Qiskit 1.0 release. This function combines ``transpile`` and ``backend.run``, which is covered by ``Sampler`` :mod:`~qiskit.primitives`. Alternatively, you can also run :func:`.transpile` followed by ``backend.run()``.
  job = execute(optimal_circuit, backend, shots=1024)
